In [1]:
from bs4 import BeautifulSoup
import requests 
import datetime
import pandas as pd 
from dateutil import parser
from dateutil.relativedelta import relativedelta

now = datetime.datetime.now()

def get_pogo_content_update_link(month_to_check):
    ''' 
    month_to_check can be current or next month 
    '''
    pogo_news_url = 'https://pokemongolive.com/en/news/'
    pogo_general_url = 'https://pokemongolive.com'

    news_page = requests.get(pogo_news_url)
    news_soup = BeautifulSoup(news_page.content, "html.parser")
    current_month_num = now.month 
    current_month = now.strftime("%B")
    next_month = (now + relativedelta(months=1)).strftime('%B')
    blogpost_links = news_soup.findAll('a', attrs = {'class':'blogList__post'})

    if month_to_check == 'current':
        for item in blogpost_links:
            if 'content-update' in item['href'] and current_month.lower() in item['href']:
                return pogo_general_url + item['href']
    if month_to_check == 'next':
        for item in blogpost_links:
            if 'content-update' in item['href'] and next_month.lower() in item['href']:
                return pogo_general_url + item['href']

link = get_pogo_content_update_link('next')
print(link)

def get_pogo_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup 

def build_pogo_df(result, names_list, dates_list):
    for i in range(len(names_list)):
        row = []
        name = names_list[i]
        row.append(name)
        date = dates_list[i]
        row.append(date)
        result.append(row)
    return result

def pogo_general_text_strip(list):
    for i in range(len(list)):
        list[i] = list[i].text.strip()
    return list 

def pogo_names_text_strip(list):
    for i in range(len(list)):
        list[i] = list[i].text.strip()
        list[i] = ''.join(filter(str.isalnum, list[i]))
    return list 

my_soup = get_pogo_soup(link)

def get_pogo_site_info(soup, info_to_get, timeframe):
    res = []
    if timeframe == 'next':
        month = (now + relativedelta(months=1)).strftime('%B')
    elif timeframe == 'current':
        month = now.strftime("%B")
    
    five_star_raid_table = soup.find(text = 'Five-Star Raids').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"MarkdownBlock__blocks"})
    five_star_names = five_star_raid_table.findAll('div', attrs = {'class':"PokemonImageGridBlock__grid__pokemon__caption"})
    five_star_names = pogo_names_text_strip(five_star_names)
    five_star_raid_dates = soup.find(text = 'Five-Star Raids').findPrevious('div', attrs = {'class':'ContainerBlock'}).findAll('strong')

    # Handle if multiple pokemon per date
    for i in range(len(five_star_raid_dates)):
        five_star_pokemon_on_site = five_star_raid_dates[i].findNext('div',attrs={'class':"block block--PokemonImageGridBlock"}).findAll('div', attrs={'class':'PokemonImageGridBlock__grid__pokemon__caption'})
        date_duplicate_count = len(five_star_pokemon_on_site)
        # if multiple pokemon for a date, duplicate that date entry in the list
        if (date_duplicate_count > 1):
            five_star_raid_dates.insert(i, five_star_raid_dates[i])
        five_star_raid_dates[i] = five_star_raid_dates[i].text.strip()

    res = build_pogo_df(res, five_star_names, five_star_raid_dates)
    df = pd.DataFrame(res, columns=["Name" ,"Dates"])

    df_prime = df.to_string(index = False, justify='center')

    mega_raid_table = soup.find(text = 'Mega Raids').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"MarkdownBlock__blocks"})
    mega_pokemon_on_site = mega_raid_table.findAll('div', attrs = {'class':"PokemonImageGridBlock__grid__pokemon__caption"})
    mega_pokemon_on_site = pogo_names_text_strip(mega_pokemon_on_site)    
    mega_raid_dates = soup.find(text = 'Mega Raids').findPrevious('div', attrs = {'class':'ContainerBlock'}).findAll('strong')
    
    # Handle if multiple pokemon per date
    for i in range(len(mega_raid_dates)):
        mega_pokemon_on_site = mega_raid_dates[i].findNext('div',attrs={'class':"block block--PokemonImageGridBlock"}).findAll('div', attrs={'class':'PokemonImageGridBlock__grid__pokemon__caption'})
        date_duplicate_count = len(mega_pokemon_on_site)
        # if multiple pokemon for a date, duplicate that date entry in the list
        if (date_duplicate_count > 1):
            mega_raid_dates.insert(i, mega_raid_dates[i])
        mega_raid_dates[i] = mega_raid_dates[i].text.strip()

    raid_hour_table = soup.find(text = 'Raid Hours').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"MarkdownBlock__blocks"})
    raid_hours_on_site = raid_hour_table.findAll('div', attrs = {'class':"PokemonImageGridBlock__grid__pokemon__caption"})
    raid_hours_on_site = pogo_general_text_strip(raid_hours_on_site)

    # research_breakthrough_table = soup.find(text = (now + relativedelta(months=1)).strftime('%B') + ' Research Breakthrough encounters').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"MarkdownBlock__blocks"})
    research_breakthrough_name = soup.find(text = month + ' Research Breakthrough encounters').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"PokemonImageGridBlock__grid__pokemon__caption"})
    research_breakthrough_dates = research_breakthrough_name.findPrevious('p').text.strip()

    spotlight_hour_table = soup.find(text = 'Pokémon Spotlight Hours').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"ContainerBlock__blocks"})
    spotlight_hours_on_site = spotlight_hour_table.findAll('div', attrs = {'class':"PokemonImageGridBlock__grid__pokemon__caption"})
    spotlight_hours_on_site = pogo_general_text_strip(spotlight_hours_on_site)

    comm_day_table = soup.find(text = month + ' Community Day ').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('p')
    comm_day_info = comm_day_table.text.strip()
    
    upcoming_events_table = soup.find(text = 'Upcoming events').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('ul')
    upcoming_events = upcoming_events_table.text.strip()

frame = get_pogo_site_info(my_soup, None, 'next')



None


MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant http://None?

In [2]:
print("Hi")

Hi


In [9]:
# moves
import requests
from bs4 import BeautifulSoup
from time import sleep
import csv
import pandas as pd
import random 
import difflib
import pokebot_parser
import numpy as np 

def get_moves_list():
    pokemon_data = pd.read_csv("pokemon_moves.csv")
    move_df = pd.DataFrame(pokemon_data)
    move_names_lst = move_df['Name']
    return move_names_lst

def lower_moves_list(move_names_list):
    for i in range(len(move_names_list)):
        move_names_list[i] = move_names_list[i].lower()
    return move_names_list
    
exceptions_lst = []

def move_name_extractor(name):
    moves_names_lst = get_moves_list()
    moves_names_lst = lower_moves_list(moves_names_lst)
    n = 1
    message = name
    game_tup = pokebot_parser.game_checker(message)
    try:
        message = message.replace(game_tup[1], "")
    except:
        pass 
    cutoff = 0.8
    message = message.replace(" ", "")
    result = [message[i: j] for i in range(len(message))
        for j in range(i + 1, len(message) + 1)]
    substring_list = []
    for substring in result:
        close_matches = difflib.get_close_matches(substring, moves_names_lst, n, cutoff)
        if (len(close_matches) > 0):
            substring_list.append(close_matches[0])
    unique, counts = np.unique(substring_list, return_counts=True)
    try:
        if (max(counts) == min(counts)):
            item_of_interest = max(unique, key=len)
        else:
            item_of_interest = unique[np.argmax(counts)]
        # print("final item is: ", item_of_interest)
        if (item_of_interest != name):
            print("above one not verified: name, item_of_interest", name, item_of_interest)
            exceptions_lst.append(name)
    except:
        final_extracted_item = 'Not Found ' + close_matches
        print(final_extracted_item)


def get_move_exceptions_lst():
    moves_names_lst = get_moves_list()
    moves_names_lst = lower_moves_list(moves_names_lst)
    for i in range(len(moves_names_lst)):
        move_name_extractor(moves_names_lst[i])
    return exceptions_lst

exc_lst = get_move_exceptions_lst()
print(exc_lst)

def exception_move_name_extractor(message, list):
    moves_names_lst = list
    moves_names_lst = lower_moves_list(moves_names_lst)
    n = 1
    # message = name
    if (message == 'quit'):
        print("QUIT")
    game_tup = pokebot_parser.game_checker(message)
    try:
        message = message.replace(game_tup[1], "")
    except:
        pass 
    cutoff = 0.95
    message = message.replace(" ", "")
    result = [message[i: j] for i in range(len(message))
        for j in range(i + 1, len(message) + 1)]
    substring_list = []
    for substring in result:
        close_matches = difflib.get_close_matches(substring, moves_names_lst, n, cutoff)
        if (len(close_matches) > 0):
            substring_list.append(close_matches[0])
    unique, counts = np.unique(substring_list, return_counts=True)
    try:
        if (max(counts) == min(counts)):
            item_of_interest = max(unique, key=len)
        else:
            item_of_interest = unique[np.argmax(counts)]
        # print("final item is: ", item_of_interest)
    except:
        final_extracted_item = 'Not Found'
        # print(final_extracted_item)

for i in range(len(exc_lst)):
    exception_move_name_extractor(exc_lst[i], exc_lst)



above one not verified: name, item_of_interest barb barrage barrage
above one not verified: name, item_of_interest bubble beam bubble
above one not verified: name, item_of_interest bug bite bite
above one not verified: name, item_of_interest charge beam charge
above one not verified: name, item_of_interest clangorous soulblaze clangorous soul
above one not verified: name, item_of_interest conversion 2 conversion
above one not verified: name, item_of_interest discharge charge
above one not verified: name, item_of_interest head charge charge
above one not verified: name, item_of_interest heal block block
above one not verified: name, item_of_interest high jump kick jump kick
above one not verified: name, item_of_interest inferno overdrive overdrive
above one not verified: name, item_of_interest misty explosion explosion
above one not verified: name, item_of_interest petal blizzard blizzard
above one not verified: name, item_of_interest powder snow powder
above one not verified: name, ite

In [6]:
# Final algorithm: extract a move name from a message 
import requests
from bs4 import BeautifulSoup
from time import sleep
import csv
import pandas as pd
import random 
import difflib
import pokebot_parser
import numpy as np 


def get_moves_list():
    pokemon_data = pd.read_csv("pokemon_moves.csv")
    move_df = pd.DataFrame(pokemon_data)
    move_names_lst = move_df['Name']
    return move_names_lst

def lower_moves_list(move_names_list):
    for i in range(len(move_names_list)):
        move_names_list[i] = move_names_list[i].lower()
    return move_names_list

def exceptions_move_name_extractor(result, check):
    moves_names_lst = get_moves_list()
    upper_moves_names_lst = get_moves_list()
    moves_names_lst = lower_moves_list(moves_names_lst).tolist()

    exceptions_lst = ['barb barrage', 'bubble beam', 'bug bite', 'charge beam', 'clangorous soulblaze', 'conversion 2', 'conversion', 'discharge', 'head charge', 'heal block', 'high jump kick', 'inferno overdrive', 'misty explosion', 'petal blizzard', 'powder snow', 'power trick',
    'psychic fangs', 'rage powder', 'reflect type', 'sleep powder', 'sludge bomb', 'sludge wave', 'splishy splash', 'steam eruption', 'strength sap', 'struggle bug', 'stun spore', 'thunder cage', 'thunder fang', 'thunder punch', 'thunder wave', 'toxic spikes', 'trick room',
    'volt tackle', 'wild charge', 'zen headbutt', 'gyro ball', 'heal block', 'high jump kick', 'mat block', 'sand tomb', 'stun spore', 'trick room', 'water sport', 'pyro ball', 'block', 'jump kick', 'block', 'sandstorm', 'spore', 'trick']

    substring_list = []
    n = 1
    cutoff = 0.95
    for substring in result:
        close_matches = difflib.get_close_matches(substring, exceptions_lst, n, cutoff)
        if (len(close_matches) > 0):
            substring_list.append(close_matches[0])
    unique, counts = np.unique(substring_list, return_counts=True)
    try:
        if (max(counts) == min(counts)):
            item_of_interest = max(unique, key=len)
            if (check == 1):
                idx = moves_names_lst.index(item_of_interest)
                final_extracted_item = upper_moves_names_lst[idx]
                return final_extracted_item 
        else:
            item_of_interest = unique[np.argmax(counts)]
            if (check == 1):
                idx = moves_names_lst.index(item_of_interest)
                final_extracted_item = upper_moves_names_lst[idx]
                return final_extracted_item 
        idx = exceptions_lst.index(item_of_interest)
        final_extracted_item = exceptions_lst[idx]
        return final_extracted_item
    except:
        final_extracted_item = 'Not Found'
        return final_extracted_item

exc_list = []

def move_name_extractor(name, check):
    moves_names_lst = get_moves_list()
    upper_moves_names_lst = get_moves_list()
    moves_names_lst = lower_moves_list(moves_names_lst).tolist()
    
    n = 1
    message = name
    game_tup = pokebot_parser.game_checker(message)
    # If there's a game in the message, should probably move this into the output file 
    try:
        message = message.replace(game_tup[1], "")
    except:
        pass 

    cutoff = 0.7
    message = message.replace(" ", "")
    result = [message[i: j] for i in range(len(message))
        for j in range(i + 1, len(message) + 1)]
    
    # Handle special cases for names, less able to handle spelling errors but not a huge deal
    final_extracted_item = exceptions_move_name_extractor(result, check)
    if (final_extracted_item != 'Not Found'):
        return final_extracted_item

    substring_list = []
    for substring in result:
        close_matches = difflib.get_close_matches(substring, moves_names_lst, n, cutoff)
        if (len(close_matches) > 0):
            substring_list.append(close_matches[0])
    unique, counts = np.unique(substring_list, return_counts=True)
    try:
        if (max(counts) == min(counts)):
            item_of_interest = max(unique, key=len)
            if (check == 1):
                idx = moves_names_lst.index(item_of_interest)
                final_extracted_item = upper_moves_names_lst[idx]
                return final_extracted_item
            return item_of_interest
        else:
            item_of_interest = unique[np.argmax(counts)]
            if (check == 1):
                idx = moves_names_lst.index(item_of_interest)
                final_extracted_item = upper_moves_names_lst[idx]
                return final_extracted_item
            return item_of_interest
    except:
        final_extracted_item = 'Not Found ' + str(close_matches)
        return final_extracted_item

def format_pokemondb_move_urls(move_name):
    general_url = 'https://pokemondb.net/move/'
    move_lst = move_name.split()
    new_name = ''
    for i in range(len(move_lst)):
        move_lst[i] = move_lst[i].lower()
    new_name = ' '.join(move_lst)
    new_name = new_name.replace(",", "")
    new_name = new_name.replace("'", "")
    new_name = new_name.replace(" ", "-")
    return general_url + new_name 

def get_move_info_csv(query, message):
    move_name = move_name_extractor(message, 1)
    data = pd.read_csv("pokemon_moves.csv")
    df = pd.DataFrame(data)
    df.set_index('Name', inplace=True)
    out = df.loc[move_name, query]
    if (str(out) == 'nan'):
        return 'No extra effects'
    return out 

while True:
    message = input()
    output = get_move_info_csv('Effect', message)
    print(output)









User receives recoil damage. May paralyze opponent.
May lower opponent's Special Defense.
High critical hit ratio.
High critical hit ratio.
User must recharge next turn.
May paralyze opponent.
User restores HP each turn. User cannot escape/switch.
User recovers half the HP inflicted on opponent.
User recovers half the HP inflicted on opponent.
User recovers half the HP inflicted on opponent.
User performs the opponent's last move.
Ignores Accuracy and Evasiveness.
User attacks for 2-3 turns but then becomes confused.
Confuses opponent.
Confuses opponent.
User sleeps for 2 turns, but user is fully healed.
Power increases with higher Friendship.
Power decreases with higher Friendship.
Pikachu-exclusive G-Max Move. Paralyzes opponents.
User recovers half of its max HP and loses the Flying type temporarily.


UnboundLocalError: local variable 'close_matches' referenced before assignment

In [36]:
# might be easier than I thought
# get url, try searching with text = 'name', if you can't find anything, then it's a no 
# otherwise it's a yes 
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from time import sleep 
import random 

data = pd.read_csv("Pokemon_data.csv")
def get_name_from_dex_number(pokemon_id):
    pokemon_df = pd.DataFrame(data)
    pokemon_df.set_index("pokedex_number", inplace = True)
    pokemon_name = pokemon_df.loc[pokemon_id, 'name']
    name_series = pd.Series(pokemon_name)
    pokemon_name = name_series.iloc[0]
    return pokemon_name

def list_of_links_generator(pokemon_id, new_number, pokemon_page, list_of_links):
    g1 = '/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g1)
    g2 = '-gs/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g2)
    g3 = '-rs/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g3)
    g4 = '-dp/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g4)
    g5 = '-bw/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g5)
    g6 = '-xy/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g6)
    g7 = '-sm/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g7)
    g8_name = get_name_from_dex_number(pokemon_id).lower()
    g8 = '-swsh/' + g8_name
    list_of_links.append(pokemon_page + g8)

    return list_of_links

number = str(254).zfill(3)
pokemon_page = 'https://www.serebii.net/pokedex'
my_links = list_of_links_generator(254, number, pokemon_page, [])
# contains links for gens 1-8 for each pokemon
# be sure to include a try/except statement for if a link doesn't even work 
def can_learn_move(links, move_name):
    gen_dict = {}
    # gen 1
    text = move_name
    gen_num = 1
    for i in range(len(links)):
        gen_link = links[i]
        print("gen link is: ", gen_link)
        sleep(random.randint(0,3))
        try:
            page = requests.get(gen_link)
            soup = BeautifulSoup(page.content, "html.parser")
            move_found = soup.find(text = move_name)
            print("move found is: ", move_found)
            if len(move_found) > 0:
                move_found = "Yes it learns it in this gen"
            else:
                move_found = "Nope"
        except:
            move_found = "Nope"
        gen_dict['Gen ' + str(gen_num)] = move_found
        gen_num += 1
    
    return gen_dict
    
test_dict = can_learn_move(my_links, "Detect")
print(test_dict)



gen link is:  https://www.serebii.net/pokedex/254.shtml
move found is:  None
gen link is:  https://www.serebii.net/pokedex-gs/254.shtml
move found is:  None
gen link is:  https://www.serebii.net/pokedex-rs/254.shtml
move found is:  Detect
gen link is:  https://www.serebii.net/pokedex-dp/254.shtml
move found is:  Detect
gen link is:  https://www.serebii.net/pokedex-bw/254.shtml
move found is:  Detect
gen link is:  https://www.serebii.net/pokedex-xy/254.shtml
move found is:  Detect
gen link is:  https://www.serebii.net/pokedex-sm/254.shtml
move found is:  Detect
gen link is:  https://www.serebii.net/pokedex-swsh/sceptile
move found is:  Detect
{'Gen 1': 'Nope', 'Gen 2': 'Nope', 'Gen 3': 'Yes it learns it in this gen', 'Gen 4': 'Yes it learns it in this gen', 'Gen 5': 'Yes it learns it in this gen', 'Gen 6': 'Yes it learns it in this gen', 'Gen 7': 'Yes it learns it in this gen', 'Gen 8': 'Yes it learns it in this gen'}


In [28]:
# Next one
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from time import sleep 
import random 


data = pd.read_csv("Pokemon_data.csv")
def get_name_from_dex_number(pokemon_id):
    pokemon_df = pd.DataFrame(data)
    pokemon_df.set_index("pokedex_number", inplace = True)
    pokemon_name = pokemon_df.loc[pokemon_id, 'name']
    name_series = pd.Series(pokemon_name)
    pokemon_name = name_series.iloc[0]
    return pokemon_name

def list_of_links_generator(pokemon_id, new_number, pokemon_page, list_of_links):
    g1 = '/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g1)
    g2 = '-gs/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g2)
    g3 = '-rs/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g3)
    g4 = '-dp/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g4)
    g5 = '-bw/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g5)
    g6 = '-xy/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g6)
    g7 = '-sm/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g7)
    g8_name = get_name_from_dex_number(pokemon_id).lower()
    g8 = '-swsh/' + g8_name
    list_of_links.append(pokemon_page + g8)

    return list_of_links

number = str(34).zfill(3)
pokemon_page = 'https://www.serebii.net/pokedex'
my_links = list_of_links_generator(34, number, pokemon_page, [])

# define a function which gets a level from a table
def get_move_level(soup, table_name, move_name):
    move_info = soup.find(text = move_name).findPrevious('tr').find('td').text.strip()
    return move_info 

def when_learn_move(links, move_name):
    move_dict = {}

    # gen 1
    gen1_link = links[0]
    gen1_page = requests.get(gen1_link)
    gen1_soup = BeautifulSoup(gen1_page.content, "html.parser")
    print(gen1_link)
    try:
        level_up = get_move_level(gen1_soup, "Generation I Level Up", move_name)
    except:
        level_up = "Can't learn by level up in gen 1"
    try:
        tm_hm = gen1_soup.find(text = 'TM & HM Attacks').findAll(text = move_name)
        tm_hm = "Can learn by TM!!"
    except:
        tm_hm = "Can't learn by TM/HM in gen 1"
    print("by level up in gen 1: ", level_up)
    print("by tm/hm in gen 1: ", tm_hm)

    sleep(random.randint(0,3))
    # gen 2
    gen2_link = links[1]
    gen2_page = requests.get(gen2_link)
    gen2_soup = BeautifulSoup(gen2_page.content, "html.parser")
    try:
        level_up = get_move_level(gen2_soup, 'Generation II Level Up', move_name)
    except:
        level_up = "Can't learn by level up in gen 2"
    print("by level up in gen 2: ", level_up)
    try:
        tm_hm = gen2_soup.find(text = 'TM & HM Attacks').findAll(text = move_name)
        if len(tm_hm) > 0:
            tm_hm = "It can learn it using a TM/HM"
        else:
            tm_hm = ""
    except:
        tm_hm = ""
    print("by tm/hm in gen 2: ", tm_hm)
    try:
        crystal_move_tutor = gen2_soup.find(text = 'Crystal Move Tutor Attacks').findAll(text = move_name)
        if len(crystal_move_tutor) > 0:
            crystal_move_tutor = "It can learn it with the move tutor"
        else:
            crystal_move_tutor = ""
    except:
        crystal_move_tutor = ""
    print("by move tutor in gen 2: ", crystal_move_tutor)
    try:
        egg_moves = gen2_soup.find(text = 'Egg Moves ').findAll(text = move_name)
        if len(egg_moves) > 0:
            egg_moves = "It can learn it as an egg move"
        else:
            egg_moves = ""
    except:
        egg_moves = ""
    print("egg moves in gen 2: ", egg_moves)

when_learn_move(my_links, "Frustration")


https://www.serebii.net/pokedex/034.shtml
by level up in gen 1:  Can't learn by level up in gen 1
by tm/hm in gen 1:  Can't learn by TM/HM in gen 1
by level up in gen 2:  Can't learn by level up in gen 2
by tm/hm in gen 2:  
by move tutor in gen 2:  
egg moves in gen 2:  


In [56]:
# New try: just call find on a page and give out any and all info on the move
# when does " " learn " " in " " 

import pandas as pd 
import requests
from bs4 import BeautifulSoup
from time import sleep 
import random 

data = pd.read_csv("Pokemon_data.csv")
def get_name_from_dex_number(pokemon_id):
    pokemon_df = pd.DataFrame(data)
    pokemon_df.set_index("pokedex_number", inplace = True)
    pokemon_name = pokemon_df.loc[pokemon_id, 'name']
    name_series = pd.Series(pokemon_name)
    pokemon_name = name_series.iloc[0]
    return pokemon_name

def list_of_links_generator(pokemon_id, new_number, pokemon_page, list_of_links):
    g1 = '/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g1)
    g2 = '-gs/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g2)
    g3 = '-rs/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g3)
    g4 = '-dp/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g4)
    g5 = '-bw/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g5)
    g6 = '-xy/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g6)
    g7 = '-sm/' + new_number + '.shtml'
    list_of_links.append(pokemon_page + g7)
    g8_name = get_name_from_dex_number(pokemon_id).lower()
    g8 = '-swsh/' + g8_name
    list_of_links.append(pokemon_page + g8)

    return list_of_links

def gen_extractor(message):
    idx = message.index("gen")
    return int(message[idx + 4])

def get_move_information(soup, move_name):
    # move_info = soup.findAll(text = move_name)[0].findPrevious('tr').find('td').text.strip()
    ret_str = ''
    move_info = soup.findAll(text = move_name)
    for item in move_info:
        final_item = item.findPrevious('tr').find('td').text.strip()
        try:
            final_item = int(final_item)
            final_item = "Level " + str(final_item)
        except:
            pass
        ret_str += final_item + '\n'
    if len(ret_str) == 0:
        ret_str = "It can't learn it"
    return ret_str 

def when_learn_move(links, move_name, message):
    number = str(146).zfill(3)
    pokemon_page = 'https://www.serebii.net/pokedex'
    my_links = list_of_links_generator(146, number, pokemon_page, [])

    idx = gen_extractor(message)
    link = links[idx - 1]
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    try:
        move_info = get_move_information(soup, move_name)
    except:
        move_info = "Error: when_learn_move"
    print(move_info)

when_learn_move(my_links, "Substitute", "When does Moltres learn sky attack in gen 1")


TM50



In [28]:
# extract a game from a message: 
import pokebot_parser
import difflib
import numpy as np 
import move_parser

def get_gen_from_game(game):
    gen_dct = {'Red':'Gen 1', 'Blue':'Gen 1', 'Yellow':'Gen 1', 'Gold':'Gen 2', 'Silver':'Gen 2', 'Crystal':'Gen 2',
    'Ruby':'Gen 3', 'Sapphire':'Gen 3', 'Emerald':'Gen 3', 'FireRed':'Gen 3', 'LeafGreen':'Gen 3',
    'Diamond':'Gen 4', 'Pearl':'Gen 4', 'Platinum':'Gen 4', 'HeartGold':'Gen 4', 'SoulSilver':'Gen 4',
    'Black':'Gen 5', 'White':'Gen 5', 'Black 2':'Gen 5', 'White 2':'Gen 5', 'X':'Gen 6', 'Y':'Gen 6', 'Omega Ruby':'Gen 6', 'Alpha Sapphire':'Gen 5',
    'Sun':'Gen 7', 'Moon':'Gen 7', 'Ultra Sun':'Gen 7', 'Ultra Moon':'Gen 7',
    'Sword':'Gen 8', 'Shield':'Gen 8', 'Brilliant Diamond':'Gen 8', 'Shining Pearl':'Gen 8', 'Legends: Arceus':'Gen 8'}
    return gen_dct[game]

message = ''

while message != 'quit':
    message = input("Enter a message: ")
    tup = pokebot_parser.game_checker(message)
    if tup[0] == True:
        game = pokebot_parser.game_processor(tup)
    else:
        game = ''
        print("no game found")
    gen = get_gen_from_game(game)
    print("Gen is: ",gen)


Gen is:  Gen 8
Gen is:  Gen 8
Gen is:  Gen 8
Gen is:  Gen 3
Gen is:  Gen 1
no game found


KeyError: ''

In [35]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from time import sleep 
import random 
import io 

# What's the next pokemon Go event? 

def dct_to_string(dct):
    ''' 
    convert a dictionary to some output string
    '''
    my_str = ''
    i = 0
    for key in dct:
        if (i < len(dct)): 
            my_str += str(dct[key]) + '\n'
        else:
            my_str += str(dct[key])
    return my_str 

def pogo_events_output():
    url = 'https://www.serebii.net/pokemongo/events.shtml'
    serebii_url = 'https://www.serebii.net/'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    latest_events = soup.find(attrs = {'class':'dextab'}).findAll('tr')[1:3]
    test_dict = {}
    final_str = 'Here are the two most recent events: \n'
    latest_events

    i = 0
    for item in latest_events:
        event_details = item.findAll('td')
        # event_details should have 3 entries, event image,name, and dates
        try:
            image_details = event_details[0].find('img')
            image = serebii_url + image_details['src']
        except:
            image = 'No image found'
        try:    
            name = event_details[1].text.strip()
        except:
            name = "No event name found"
        try:
            dates = event_details[2].text.strip()
        except:
            dates = "No dates range found"
        test_dict['image' + str(i)] = image 
        test_dict['name' + str(i)] = name
        test_dict['dates' + str(i)] = dates
        i += 1
        # final_str += dct_to_string(test_dict)
    return test_dict

dct = pogo_events_output()
print(dct)

{'image0': 'https://www.serebii.net//pokemongo/pokemongo.jpg', 'name0': 'Fashion Week 2022', 'dates0': 'September 27th - October 3rd 2022', 'image1': 'https://www.serebii.net//pokemongo/testyourmettle.jpg', 'name1': 'Test Your Mettle', 'dates1': 'September 16th - September 21st 2022'}


In [89]:
# Community Day
# this month and next month

from bs4 import BeautifulSoup
import requests 
import datetime
import pandas as pd 
from dateutil import parser
from dateutil.relativedelta import relativedelta
import pokebot_parser


# When is the community day this month? 
# When is the community day next month? 
# When is the next community day? 

today = datetime.datetime.now()

def has_passed(community_day):
    # need to remove ending of day
    community_day = community_day.split(" ")
    community_day[1] = ''.join(filter(str.isdigit, community_day[1]))

    community_day = " ".join(community_day)
    community_day_object = datetime.datetime.strptime(community_day,'%B %d %Y')

    return not (today <= community_day_object)

def get_comm_day_info(check_next):
    url = 'https://www.serebii.net/pokemongo/communityday.shtml'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    current_month_year = today.strftime("%B %Y")
    comm_day_url = url = 'https://www.serebii.net/pokemongo/communityday/' + str(current_month_year.lower().replace(" ", "")) +'.shtml'
    
    next_month_year = (today + relativedelta(months=1)).strftime('%B %Y')

    current_month_comm_day = soup.find(text = current_month_year).findNext('td').text.strip()
    name = soup.find(text = current_month_year).findNext('td').findNext('td').text.strip()

    if (check_next):
        if (has_passed(current_month_comm_day)):
            try:
                # it's passed and has been announced
                next_comm_day = soup.find(text = next_month_year).findNext('td').text.strip()
                name = soup.find(text = next_month_year).findNext('td').findNext('td').text.strip()
                ret_str = name + '\n' + next_comm_day
                return ret_str 
            except:
                # it's passed but haven't announced it yet
                ret_str = "I'm not sure, I can just find the one this month: \n" + name + '\n' + current_month_comm_day
                return ret_str 
        else:
            # hasn't passed yet
            next_comm_day = current_month_comm_day
            ret_str = name + '\n' + next_comm_day
            return ret_str 
    else:
        ret_str = name + '\n' + current_month_comm_day
        return ret_str  

item = get_comm_day_info(check_next = True)
print(item)

Litwick
October 15th 2022


In [78]:
# raids 
import requests
from bs4 import BeautifulSoup
from time import sleep
import csv
import pandas as pd
import random 
import difflib
import pokebot_parser
import numpy as np 

def get_star_search_string(message):
    raids_star_lst = ['5 star', '5-star', 'five star', 'mega raids', '4-star', '4 star', 'four star', 
    '3-star', '3 star', 'three star', '2-star', '2 star', 'two star', '1-star', '1 star', 'one star']
    # output closest match

    raids_dct = {'5 star':'☆☆☆☆☆ List', '5-star':'☆☆☆☆☆ List', 'five star':'☆☆☆☆☆ List', 'mega raids':'Mega Raid List', '4-star':'☆☆☆☆ List', '4 star':'☆☆☆☆ List', 'four star':'☆☆☆☆ List', 
    '3-star':'☆☆☆ List', '3 star':'☆☆☆ List', 'three star':'☆☆☆ List', '2-star':'☆☆ List', '2 star':'☆☆ List', 'two star':'☆☆ List', '1-star':'☆ List', '1 star':'☆ List', 'one star':'☆ List'}
    # use closest match to get search string from raids_dct

    n = 1
    cutoff = 0.9
    message = message.replace(" ", "")
    result = [message[i: j] for i in range(len(message))
        for j in range(i + 1, len(message) + 1)]

    substring_list = []
    for substring in result:
        close_matches = difflib.get_close_matches(substring, raids_star_lst, n, cutoff)
        if (len(close_matches) > 0):
            substring_list.append(close_matches[0])
    key = substring_list[0]

    search_string = raids_dct[key]
    return search_string
    # use search string to get pokemon for each raid type

def get_current_raid_info(message):
    search_string = get_star_search_string(message)
    raid_url = 'https://www.serebii.net/pokemongo/raidbattles.shtml'
    page = requests.get(raid_url)
    soup = BeautifulSoup(page.content, "html.parser")
    try:
        table = soup.find(text = search_string).findNext('table', attrs = {'class':'dextab'}).find_all('tr')[1:]
    except:
        return "I'm not sure"
    ret_str = ''
    for element in table: 
        lst_num = element.findAll('td', attrs = {'class':'cen'})
        lst_name = element.findAll('td', attrs = {'class':'fooinfo'})
        if (len(lst_num) > 0):
            num = lst_num[0].text.strip()
            ret_str += num + ' | '
        if (len(lst_name) > 0):
            try:
                extra_info = lst_name[0].find('i').text.strip()
                name = lst_name[0].find('a').text.strip()
                ret_str += name + ' | '
                ret_str += extra_info + '\n'
            except:
                name = lst_name[0].find('a').text.strip()
                ret_str += name + '\n' 
    return ret_str 

message = input("Ask about any current raid tiers you're curious about: ")
item = get_current_raid_info(message)
print(item)

#797 | Celesteela | Southern Hemisphere
#798 | Kartana | Northern Hemisphere



In [18]:
# spotlight hour:
#NOTE: FIX THIS SO CAN HANDLE CURRENT

# what are the spotlight hours this month? 
# what's the next spotlight hour? 

# take today, and compare to most recent 4 days on table and see which is the first upcoming one

from bs4 import BeautifulSoup
import requests 
import datetime
import pandas as pd 
from dateutil import parser
from dateutil.relativedelta import relativedelta

today = datetime.datetime.now()

def has_passed(community_day):
    # need to remove ending of day
    community_day = community_day.split(" ")
    community_day[1] = ''.join(filter(str.isdigit, community_day[1]))

    community_day = " ".join(community_day)
    community_day_object = datetime.datetime.strptime(community_day,'%B %d %Y')

    return not (today <= community_day_object)

def find_next(x, sorted_dates_lst):
    y = [a for a in sorted_dates_lst if a > today]
    # return y[0].strftime('%B %d %Y') if y else None
    return y[0] if y else None

def get_spotlight_hour_info(check_next):
    url = 'https://www.serebii.net/pokemongo/events/spotlighthour.shtml'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    table = soup.find('table', attrs = {'class':'dextab'})
    dates = table.findAll('tr')[1:9]
    dates_lst = []
    dates_obj_lst = []
    pokemon_lst = []
    effects_lst = []

    for date in dates:
        final_date = date.findAll('td')[0].text.strip()
        pokemon = date.findAll('td')[1].text.strip()
        effect = date.findAll('td')[2].text.strip()

        dates_lst.append(final_date)
        pokemon_lst.append(pokemon)
        effects_lst.append(effect)
    
    if (check_next == False):
        ret_str = 'Here are some of the most recent ones I could find: \n'
        for i in range(len(dates_lst)):
            date = dates_lst[i]
            name = pokemon_lst[i]
            effect = effects_lst[i]
            if (i < len(dates_lst) - 1):
                ret_str += date + " | " + name + " | " + effect + '\n' 
            else:
                ret_str += date + " | " + name + " | " + effect
        return ret_str 

    for date in dates_lst:
        date = date.split(" ")
        date[1] = ''.join(filter(str.isdigit, date[1]))
        date = " ".join(date)
        date_obj = datetime.datetime.strptime(date,'%B %d %Y')
        dates_obj_lst.append(date_obj)

    # https://stackoverflow.com/questions/66165563/given-a-specific-date-find-the-next-date-in-the-list-using-python
    dates_obj_lst.sort()
    res = find_next(today, dates_obj_lst)
    idx = dates_obj_lst.index(res) 
    res = res.strftime('%B %d %Y')
    final_str = "The next spotlight hour is: \n" + res + ' | ' + pokemon_lst[idx] + ' | ' + effects_lst[idx]

    return final_str  

item = get_spotlight_hour_info(True)
print(item)



The next spotlight hour is: 
September 27 2022 | Nidoran♀ | 2 * Catch Experience


In [74]:
# research breakthrough
from bs4 import BeautifulSoup
import requests 

# Searches page for most recent research breakthrough
def get_most_recent_research_breakthroughs():
    url = 'https://www.serebii.net/pokemongo/fieldresearch.shtml'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    try:
        research_breakthrough = soup.find(text = "Field Research List").findPrevious('p').findNext('div').find('li')
        date = research_breakthrough.get("title")
        pokemon = research_breakthrough.find('td', attrs = {'class':'pkmn'}).findNext('td').text.strip()
        ret_str = "Here's the most recent one I could find: \n" + date + " | " + pokemon
        return ret_str 
    except:
        ret_str = "I'm not sure!"
        return ret_str 
    
item = get_most_recent_research_breakthroughs()
print(item)


Here's the most recent one I could find: 
September 2022 | Medicham Encounter


In [ ]:
# NOTE: OLD CODE HERE
def get_pogo_content_update_link(month_to_check):
    ''' 
    month_to_check can be current or next month 
    '''
    pogo_news_url = 'https://pokemongolive.com/en/news/'
    pogo_general_url = 'https://pokemongolive.com'

    news_page = requests.get(pogo_news_url)
    news_soup = BeautifulSoup(news_page.content, "html.parser")
    current_month_num = today.month 
    current_month = today.strftime("%B")
    next_month = (today + relativedelta(months=1)).strftime('%B')
    blogpost_links = news_soup.findAll('a', attrs = {'class':'blogList__post'})
    print(blogpost_links)

    if month_to_check == 'current':
        for item in blogpost_links:
            if 'content-update' in item['href'] and current_month.lower() in item['href']:
                return pogo_general_url + item['href']
    if month_to_check == 'next':
        for item in blogpost_links:
            if 'content-update' in item['href'] and next_month.lower() in item['href']:
                return pogo_general_url + item['href']

# link = get_pogo_content_update_link('next')
# print(link)

def get_pogo_soup(url):
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        return soup
    except:
        return 'Not Found'

def build_pogo_df(result, names_list, dates_list):
    for i in range(len(names_list)):
        row = []
        name = names_list[i]
        row.append(name)
        date = dates_list[i]
        row.append(date)
        result.append(row)
    return result

def build_pogo_df_general(result, list):
    for i in range(len(list)):
        row = []
        item = list[i]
        row.append(item)        
        result.append(row)
    return result 

def pogo_general_text_strip(list):
    for i in range(len(list)):
        list[i] = list[i].text.strip()
    return list 

def pogo_names_text_strip(list):
    for i in range(len(list)):
        list[i] = list[i].text.strip()
        list[i] = ''.join(filter(str.isalnum, list[i]))
    return list 

# my_soup = get_pogo_soup(link)

def get_pogo_site_info(soup, info_to_get, timeframe):
    res = []
    if timeframe == 'next':
        month = (today + relativedelta(months=1)).strftime('%B')
    elif timeframe == 'current':
        month = today.strftime("%B")
    
    five_star_raid_table = soup.find(text = 'Five-Star Raids').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"MarkdownBlock__blocks"})
    five_star_names = five_star_raid_table.findAll('div', attrs = {'class':"PokemonImageGridBlock__grid__pokemon__caption"})
    five_star_names = pogo_names_text_strip(five_star_names)
    five_star_raid_dates = soup.find(text = 'Five-Star Raids').findPrevious('div', attrs = {'class':'ContainerBlock'}).findAll('strong')

    # Handle if multiple pokemon per date
    for i in range(len(five_star_raid_dates)):
        five_star_pokemon_on_site = five_star_raid_dates[i].findNext('div',attrs={'class':"block block--PokemonImageGridBlock"}).findAll('div', attrs={'class':'PokemonImageGridBlock__grid__pokemon__caption'})
        date_duplicate_count = len(five_star_pokemon_on_site)
        # if multiple pokemon for a date, duplicate that date entry in the list
        if (date_duplicate_count > 1):
            five_star_raid_dates.insert(i, five_star_raid_dates[i])
        five_star_raid_dates[i] = five_star_raid_dates[i].text.strip()

    if (info_to_get == '5-star raids'):
        res = build_pogo_df(res, five_star_names, five_star_raid_dates)
        df = pd.DataFrame(res, columns=["Name" ,"Dates"])
        df_prime = df.to_string(index = False, justify='left', header=False)
        return df_prime

    mega_raid_table = soup.find(text = 'Mega Raids').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"MarkdownBlock__blocks"})
    mega_pokemon_on_site = mega_raid_table.findAll('div', attrs = {'class':"PokemonImageGridBlock__grid__pokemon__caption"})
    mega_pokemon_on_site = pogo_names_text_strip(mega_pokemon_on_site)    
    mega_raid_dates = soup.find(text = 'Mega Raids').findPrevious('div', attrs = {'class':'ContainerBlock'}).findAll('strong')
    
    # Handle if multiple pokemon per date
    for i in range(len(mega_raid_dates)):
        mega_pokemon_on_site = mega_raid_dates[i].findNext('div',attrs={'class':"block block--PokemonImageGridBlock"}).findAll('div', attrs={'class':'PokemonImageGridBlock__grid__pokemon__caption'})
        date_duplicate_count = len(mega_pokemon_on_site)
        # if multiple pokemon for a date, duplicate that date entry in the list
        if (date_duplicate_count > 1):
            mega_raid_dates.insert(i, mega_raid_dates[i])
        mega_raid_dates[i] = mega_raid_dates[i].text.strip()
    
    if (info_to_get == 'mega raids'):
        res = build_pogo_df(res, mega_pokemon_on_site, mega_raid_dates)
        df = pd.DataFrame(res, columns=["Name" ,"Dates"])
        df_prime = df.to_string(index = False, justify='left', header=False)
        return df_prime

    raid_hour_table = soup.find(text = 'Raid Hours').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"MarkdownBlock__blocks"})
    raid_hours_on_site = raid_hour_table.findAll('div', attrs = {'class':"PokemonImageGridBlock__grid__pokemon__caption"})
    raid_hours_on_site = pogo_general_text_strip(raid_hours_on_site)

    if (info_to_get == 'raid hour'):
        res = build_pogo_df_general(res, raid_hours_on_site)
        df = pd.DataFrame(res, columns=["Raid Hour"])
        df_prime = df.to_string(index = False, justify='left', header=False)
        return df_prime

    # research_breakthrough_table = soup.find(text = (now + relativedelta(months=1)).strftime('%B') + ' Research Breakthrough encounters').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"MarkdownBlock__blocks"})
    research_breakthrough_name = soup.find(text = month + ' Research Breakthrough encounters').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"PokemonImageGridBlock__grid__pokemon__caption"})
    research_breakthrough_dates = research_breakthrough_name.findPrevious('p').text.strip()
    
    if (info_to_get == 'research breakthrough'):
        row = []
        row.append(research_breakthrough_name)
        row.append(research_breakthrough_dates)
        res.append(row)
        df = pd.DataFrame(res, columns=["Name", "Date"])
        df_prime = df.to_string(index = False, justify='left', header=False)
        return df_prime

    spotlight_hour_table = soup.find(text = 'Pokémon Spotlight Hours').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('div', attrs = {'class':"ContainerBlock__blocks"})
    spotlight_hours_on_site = spotlight_hour_table.findAll('div', attrs = {'class':"PokemonImageGridBlock__grid__pokemon__caption"})
    spotlight_hours_on_site = pogo_general_text_strip(spotlight_hours_on_site)

    if (info_to_get == 'spotlight hour'):
        res = build_pogo_df_general(res, spotlight_hours_on_site)
        df = pd.DataFrame(res, columns=["Spotlight Hour"])
        df_prime = df.to_string(index = False, justify='left', header=False)
        return df_prime

    comm_day_table = soup.find(text = month + ' Community Day ').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('p')
    comm_day_info = comm_day_table.text.strip()

    if (info_to_get == 'comm day'):
        return comm_day_info
    
    upcoming_events_table = soup.find(text = 'Upcoming events').findPrevious('div', attrs = {'class':'ContainerBlock'}).find('ul')
    upcoming_events = upcoming_events_table.text.strip()

    if (info_to_get == 'upcoming events'):
        return upcoming_events
